In [19]:
# packages
import os 
import sys
import random
import numpy as np 
from numpy import linalg as LA
from time import process_time

In [20]:
# method to read data from file
# OUTPUT: adjacency matrix
def read(filename):
    PROJECT_PATH = "./DIMACS/"
    path_to_file = os.path.join(PROJECT_PATH, filename)
    
    with open(path_to_file, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                line.split()[1:]
                #print(*line.split()[1:])
            # first line: p name num_of_vertices num_of_edges
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                print('{0} {1} {2}\n'.format(name, vertices_num, edges_num))
                adj = np.zeros((int(vertices_num),int(vertices_num)))
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                adj[int(v1)-1][int(v2)-1]=1
                adj[int(v2)-1][int(v1)-1]=1
            else:
                continue
                
        #return adjacency matrix        
        return int(vertices_num), adj  

In [21]:
num, adj=read("c125.9.clq") #Take a look at the graph size/ adjacency

col 125 6963



COMPLEMENTARY METHODS:

In [43]:
# output the function value of 0.5*x'Ax
def func_val(x, Q):
    val = 0.5*x.transpose().dot(Q).dot(x)
    return val

# check the final result if it is indeed a clique
def check(adj,index):
    for i in index:
        for j in index:
            if i!=j and adj[i][j]!=1:
                return "false"
            
    return "true"

MAIN ALGORITHM:

In [56]:
#Armijo Backtracking
def case_3(vertice_num, x, adj, max_iters, delta, mu, epsilon):
    # Modify the adjacency matrix
    Id = np.identity(vertice_num)
    Q = -(adj+1/2*Id)
    L = np.sqrt(np.trace(Q.dot(Q)))     # L=root(trace(A'A))
    p = 4
    # p=3 does not work but it works with p=-1 ?
    
    for iteration in range(max_iters):
        
##1     # Hessian derived from entropy function
        hessian = p*(p-1)*np.diag((x+epsilon)**(p-2))

    
##2     # step direction
        Qx = Q.dot(x) ; x_p = (x+epsilon)**(2-p)
        v_x = -x_p*(Qx - x_p.dot(Qx)/sum(x_p))/(p*(p-1))

    
##3     # step-size
        #--descent inequality
        vHv = p*(p-1)*(v_x/x_p).dot(v_x)
        alpha = 0.95*(2*vHv)/(L*v_x.dot(v_x))
        
##4     # set test point        
        x_hat = x + alpha*v_x
    
        if x_hat.min()<0:
            np.min([x[i]/(-v_x[i]) for i in range(vertice_num) if v_x[i]<0]) # alpha_0= min(x_i/(-v_i) for v<0)

            
##5     # sufficient decrease
        f_val = func_val(x, Q)
        f_add = mu*vHv
        while func_val(x_hat, Q) > f_val - alpha*f_add:
            alpha = alpha*delta     #shrink step size
            x_hat = x + alpha*v_x     # update test point

##6     # stopping Rule 
        if (LA.norm(x_hat-x)/LA.norm(x))<(epsilon):           
            print("Reached optimality after {} iterations\n".format(iteration))
            break

##7     # new state
        old_x = x   
        x = x_hat   
        
        print(func_val(x,Q))

    return x

INITIALIZE X, RUN ALGORITHM THEN OUTPUT FINAL RESULT

In [57]:
def solve(filename):
    # read the adjacency matrix 
    vertice_num, adj = read(filename)
    print("SOLUTION: ")
    
    # choose starting point at the middle of the simplex
    x=np.array([1/vertice_num for i in range(vertice_num)]) 
    
    # RUN ALGORITHM 
    # max 10^6 iteration, shrinking factor delta=0.5, mu=0.001, epsilon=10^(-9) chosen
    start = process_time()
    sol = case_3(vertice_num, x, adj, 1000000, 0.5, 0.01, 0.000000001)
    end = process_time()
    
    # retrieve the index of chosen vertices
    ans = np.array([i for i in sol if i>0.1/len(sol)])
    index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
    
    print(' '.join(str(x+1) for x in index))
    print("Maximum clique size: {}\n".format(len(ans)))
    print("Clique: {}".format(check(adj,index)))
    print('CPU = {}'.format(np.mean(end-start)))
    
    return sol

SOLVE HERE

In [58]:
solution = solve("c125.9.clq")  # epsilon = 10^(-9)

# the function value of each iteration is shown

col 125 6963

SOLUTION: 
-5531079576264.399
-43419377083540.06
-340015661141244.1
-2669122581542372.0
-2.090738544435514e+16
-1.6411906319732877e+17
-1.28583567468764e+18
-1.0093245805723668e+19
-7.909266791424154e+19
-6.208200630930021e+20
-4.865607107244634e+21
-3.819002062828082e+22
-2.9934817442929413e+23
-2.3494854200428325e+24
-1.841816554545492e+25
-1.4455270624367316e+26
-1.1332865865464067e+27
-8.894141718606598e+27
-6.9735142968948085e+28
-5.4726961204760206e+29
-4.291195594988803e+30
-3.3675517289279627e+31
-2.6406847587866197e+32
-2.0722391002578487e+33
-1.6250400577471098e+34
-1.2751924079769064e+35
-1.0000437185277209e+36
-7.847294362626923e+36
-6.154316655719747e+37
-4.829153653938346e+38
-3.787437411844955e+39
-2.9718558463271674e+40
-2.3308533125164993e+41
-1.8288964036918832e+42
-1.434457005043731e+43
-1.125522870162827e+44
-8.828006672877583e+44
-6.926642022494501e+45
-5.433001437871659e+46
-4.262788949891863e+47
-3.343633402076157e+48
-2.6234159284126854e+49
-2.0577

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in square
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


...

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

In [174]:
# RUN WITH MULTIPLE STARTING POINTS 

def multi_solve(filename, trials):
    # input the adjacency matrix
    vertice_num, adj = read(filename)

    
    #running trials of different starting x
    print('{} trials:'.format(trials)) 
    solutions = []; time = []
    Id = np.identity(vertice_num)
    A= -(adj+1/2*Id)
    
    for i in range(trials):
        print("process: {}".format(i+1))
        # choose the starting point at random
        x=np.random.dirichlet(np.ones(vertice_num)).ravel()
 
        # run algorithm
        start = process_time()
        sol = case_1(x, adj, 1000000, 0.5, 0.01, 0.000000001)
        end = process_time()
    
        # retrieve the index of the chosen vertices
        ans = np.array([i for i in sol if i>0.1/len(sol)]) 
        index = np.array([i for i in range(len(sol)) if sol[i]>0.1/len(sol)])
        
        if check(adj,index):
            solutions.append(len(ans))
            time.append(end-start)
    
    return np.array(solutions), np.array(time)

In [ ]:
solutions,times = multi_solve("c125.9.clq", 100)

col 125 6963

100 trials:
process: 1
Reached optimality after 761 iterations

process: 2
Reached optimality after 967 iterations

process: 3
Reached optimality after 545 iterations

process: 4
Reached optimality after 537 iterations

process: 5
Reached optimality after 469 iterations

process: 6
Reached optimality after 558 iterations

process: 7
Reached optimality after 725 iterations

process: 8
Reached optimality after 559 iterations

process: 9
Reached optimality after 564 iterations

process: 10
Reached optimality after 665 iterations

process: 11
Reached optimality after 803 iterations

process: 12
Reached optimality after 488 iterations

process: 13
Reached optimality after 536 iterations

process: 14
Reached optimality after 863 iterations

process: 15
Reached optimality after 633 iterations

process: 16
Reached optimality after 660 iterations

process: 17
Reached optimality after 425 iterations

process: 18
Reached optimality after 1042 iterations

process: 19


In [147]:
print('{} return true clique'.format(len(solutions)))
print('max = {}'.format(np.amax(solutions)))
print('mean = {}'.format(np.mean(solutions)))
print('std = {}'.format(np.std(solutions)))
print('CPU = {}'.format(np.mean(times)))

100 return true clique
max = 34
mean = 29.11
std = 1.765757627762089
CPU = 1.1065625
